In [1]:
from pyswip import Prolog
import pandas as pd

ImportError: Could not find the SWI-Prolog library in this platform. If you are sure it is installed, please open an issue.

In [6]:
pip install pyswip

Note: you may need to restart the kernel to use updated packages.


In [2]:
def sanatize(string):
    return string.lower().replace(" ", "_").replace('\'', '').replace('.', '').split(',')[0]

In [3]:
prolog = Prolog()

In [4]:
map_df = pd.read_csv('Adjacency_matrix.csv', index_col="Destinations")

map_df

,Tokyo,Ottawa,Mexico City,Rome,Brasilia,Canberra,New Delhi,Pretoria,Madrid,Moscow,...,Bursa,Munich,Hamburg,Frankfurt,Alexandria,Luxor,Aswan,Christchurch,Queenstown,Rotorua
Destinations,,,,,,,,,,,,,,,,,,,,,
Tokyo,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,1
Ottawa,1,0,1,0,0,1,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
Mexico City,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
Rome,1,0,0,0,1,1,1,0,0,0,...,1,0,1,0,0,0,0,0,1,0
Brasilia,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Luxor,1,1,0,0,0,0,0,0,1,0,...,0,0,1,0,0,1,0,0,1,0
Aswan,1,0,1,0,0,0,1,0,0,1,...,0,0,0,0,0,0,1,0,0,0
Christchurch,1,0,0,1,0,0,0,0,0,1,...,0,1,1,1,0,0,0,0,1,1


In [ ]:
prolog.retractall("directly_connected(_, _)")
prolog.retractall("connected(_, _)")
prolog.retractall("connected2(_, _)")
prolog.retractall("connected3(_, _)")
prolog.retractall("connected4(_, _)")

all_cities = map_df.index

visited = set()

for ct1 in all_cities:
    for ct2 in all_cities:
        if ct2 == ct1:
            continue
        if map_df[ct1][ct2] and (ct1, ct2) not in visited:
            prolog.assertz(f"directly_connected('{sanatize(ct1)}', '{sanatize(ct2)}')")
            print(f"directly_connected('{sanatize(ct1)}', '{sanatize(ct2)}')")
            visited.add((ct1, ct2))
            visited.add((ct2, ct1))

prolog.assertz("connected(X, Y) :- directly_connected(X, Y)")
prolog.assertz("connected(X, Y) :- directly_connected(Y, X)")

prolog.assertz("connected2(X, Y) :- connected(X, Z), connected(Z, Y)")

prolog.assertz("connected3(X, Y) :- connected2(X, Z), connected(Z, Y)")

prolog.assertz("connected4(X, Y) :- connected2(X, Z), connected2(Z, Y)")

In [ ]:
# # result2 = prolog.query("connected(ottawa, X), connected(X, rome)")
# # result2 = prolog.query("connected(rome, X), connected(X, yazd)")
# # result3 = prolog.query("connected2(rome, X), connected(X, yazd)")
# # for city in all_cities:
# # print(city)
# result = prolog.query(f"connected(rome, lima), connected(lima, cairo), connected(cairo, lisbon)")

# cities2 = set()

# cities = set()
# count = 0
# for res in result:
#     print(res)
#     count += 1
#     cities.add(res['X2'])

# for ct in cities:
#     result2 = prolog.query(f"connected('{ct}', Y)")
    
#     for res2 in result2:
#         cities2.add(res2['Y'])

# print("Cities:" , cities)
# print("Len:", len(cities))
# print("Cities2:" , cities2)
# print("Len2:", len(cities2))
# print("Count:", count)

In [ ]:
def result_exists(res_list):
    return len(list(res_list)) > 0

In [ ]:
def check_conn4():
    result = prolog.query(f"connected('{cities[0]}', '{cities[1]}')")

In [ ]:
print(trial(["rome", "mexico_city"]))

In [98]:
def trial(cities):
    varc = 0
    removec = 0

    total_ok_paths = []

    while len(cities) - removec > 0:
        if removec == 0:
            ok_paths = check_cities(cities)

            if len(ok_paths) > 0:
                total_ok_paths.append(ok_paths)
        elif removec == 1:
            for c in cities:
                ok_paths = check_cities([city for city in cities if city != c])

                if len(ok_paths) > 0:
                    total_ok_paths.append(ok_paths)
        elif removec == 2:
            for c1 in cities:
                for c2 in cities:
                    if c1 == ct2:
                        continue
    
                    ok_paths = check_cities([city for city in cities if city != c1 and city != c2])

                    if len(ok_paths) > 0:
                        total_ok_paths.append(ok_paths)

        if len(total_ok_paths) > 0:
            print("trial_total_ok_paths:", total_ok_paths)
            return total_ok_paths
        removec += 1

In [100]:
def check_cities(cities):
    varc = 0
    total_ok_paths = []

    while varc + len(cities) <= 4:
        print("Trying varc:", varc, cities)
        ok_paths = check_cities_varc(cities, varc)

        print("ok_paths:", ok_paths)
        if len(ok_paths) > 0:
            total_ok_paths.append(ok_paths)
    
        if len(total_ok_paths) > 0:
            print("check_cities_total_ok_paths:", total_ok_paths)
            return total_ok_paths
    
        varc += 1
    return []

In [102]:
def check_cities_varc(cities, varc):
    paths = get_permutation(cities, varc)

    print("permutation paths:", paths)
    
    ok_paths = []
    for p in paths:
        print("p:", p)
        res = check_connection(p)
        if res is not None:
            ok_paths.append((p, res))


    return ok_paths

In [104]:
def get_permutation(cities, varc = 0):
    res = []
    if len(cities) == 1:
        return [cities]

    for i in range(len(cities)):
        for j in range(len(cities)):
            if i == j:
                continue

            src = cities[i]
            dest = cities[j]
            cnt = len(cities)

            rem = [x for xi, x in enumerate(cities) if xi != i and xi != j]
            print("rem:", rem)

            if cnt == 2:
                if varc == 0:
                    res.append((src, dest))
                elif varc == 1:
                    res.append((src, 'X1', dest))
                elif varc == 2:
                    res.append((src, 'X1', 'X2', dest))
            if cnt == 3:
                if varc == 0:
                    res.append((src, dest))
                elif varc == 1:
                    res.append((src, 'X1', rem[0], dest))
                    res.append((src, rem[0], 'X1', dest))
            if cnt == 4:
                if varc == 0:
                    res.append((src, rem[0], rem[1], dest))
                    res.append((src, rem[1], rem[0], dest))
                

    return res

In [77]:
def check_connection(cities):
    if len(cities) <= 1:
        return [{}]

    prev = None
    query = ""
    for i, ct in enumerate(cities):
        if i == 0:
            prev = ct
            continue

        query += f"connected({prev}, {ct})"
        prev = ct
        if i != len(cities) - 1:
            query += ", "
        
    print(query)
    result = prolog.query(query)

    res_list = list(result)
    print(res_list)
    if result_exists(res_list):
        return res_list

    return None

In [37]:
ls = [[[(('rome', 'X1', 'mexico_city'), [{'X1': 'lima'}, {'X1': 'kuala_lumpur'}, {'X1': 'london'}, {'X1': 'copenhagen'}, {'X1': 'bern'}, {'X1': 'saint_petersburg'}, {'X1': 'karachi'}, {'X1': 'canberra'}, {'X1': 'stockholm'}, {'X1': 'ankara'}, {'X1': 'lisbon'}, {'X1': 'cancun'}, {'X1': 'lyon'}, {'X1': 'cusco'}, {'X1': 'bursa'}, {'X1': 'hamburg'}, {'X1': 'queenstown'}]), (('mexico_city', 'X1', 'rome'), [{'X1': 'canberra'}, {'X1': 'ankara'}, {'X1': 'lima'}, {'X1': 'kuala_lumpur'}, {'X1': 'london'}, {'X1': 'copenhagen'}, {'X1': 'cancun'}, {'X1': 'cusco'}, {'X1': 'bursa'}, {'X1': 'hamburg'}, {'X1': 'queenstown'}, {'X1': 'stockholm'}, {'X1': 'lisbon'}, {'X1': 'bern'}, {'X1': 'saint_petersburg'}, {'X1': 'lyon'}, {'X1': 'karachi'}])]]]
[[[(('rome', 'X1', 'mexico_city'), [{'X1': 'lima'}, {'X1': 'kuala_lumpur'}, {'X1': 'london'}, {'X1': 'copenhagen'}, {'X1': 'bern'}, {'X1': 'saint_petersburg'}, {'X1': 'karachi'}, {'X1': 'canberra'}, {'X1': 'stockholm'}, {'X1': 'ankara'}, {'X1': 'lisbon'}, {'X1': 'cancun'}, {'X1': 'lyon'}, {'X1': 'cusco'}, {'X1': 'bursa'}, {'X1': 'hamburg'}, {'X1': 'queenstown'}]), (('mexico_city', 'X1', 'rome'), [{'X1': 'canberra'}, {'X1': 'ankara'}, {'X1': 'lima'}, {'X1': 'kuala_lumpur'}, {'X1': 'london'}, {'X1': 'copenhagen'}, {'X1': 'cancun'}, {'X1': 'cusco'}, {'X1': 'bursa'}, {'X1': 'hamburg'}, {'X1': 'queenstown'}, {'X1': 'stockholm'}, {'X1': 'lisbon'}, {'X1': 'bern'}, {'X1': 'saint_petersburg'}, {'X1': 'lyon'}, {'X1': 'karachi'}])]]]
res = ls[0][0]

In [38]:
res

[(('rome', 'X1', 'mexico_city'),
  [{'X1': 'lima'},
   {'X1': 'kuala_lumpur'},
   {'X1': 'london'},
   {'X1': 'copenhagen'},
   {'X1': 'bern'},
   {'X1': 'saint_petersburg'},
   {'X1': 'karachi'},
   {'X1': 'canberra'},
   {'X1': 'stockholm'},
   {'X1': 'ankara'},
   {'X1': 'lisbon'},
   {'X1': 'cancun'},
   {'X1': 'lyon'},
   {'X1': 'cusco'},
   {'X1': 'bursa'},
   {'X1': 'hamburg'},
   {'X1': 'queenstown'}]),
 (('mexico_city', 'X1', 'rome'),
  [{'X1': 'canberra'},
   {'X1': 'ankara'},
   {'X1': 'lima'},
   {'X1': 'kuala_lumpur'},
   {'X1': 'london'},
   {'X1': 'copenhagen'},
   {'X1': 'cancun'},
   {'X1': 'cusco'},
   {'X1': 'bursa'},
   {'X1': 'hamburg'},
   {'X1': 'queenstown'},
   {'X1': 'stockholm'},
   {'X1': 'lisbon'},
   {'X1': 'bern'},
   {'X1': 'saint_petersburg'},
   {'X1': 'lyon'},
   {'X1': 'karachi'}])]

In [39]:
results = []
for type in res:
    pattern = str(type[0])[1:-1]
    print(pattern)
    for object in type[1]:
        for var in object:
            val = '%s' % pattern
            results.append(val.replace(var, object[var])) 

'rome', 'X1', 'mexico_city'
'mexico_city', 'X1', 'rome'


In [40]:
results

["'rome', 'lima', 'mexico_city'",
 "'rome', 'kuala_lumpur', 'mexico_city'",
 "'rome', 'london', 'mexico_city'",
 "'rome', 'copenhagen', 'mexico_city'",
 "'rome', 'bern', 'mexico_city'",
 "'rome', 'saint_petersburg', 'mexico_city'",
 "'rome', 'karachi', 'mexico_city'",
 "'rome', 'canberra', 'mexico_city'",
 "'rome', 'stockholm', 'mexico_city'",
 "'rome', 'ankara', 'mexico_city'",
 "'rome', 'lisbon', 'mexico_city'",
 "'rome', 'cancun', 'mexico_city'",
 "'rome', 'lyon', 'mexico_city'",
 "'rome', 'cusco', 'mexico_city'",
 "'rome', 'bursa', 'mexico_city'",
 "'rome', 'hamburg', 'mexico_city'",
 "'rome', 'queenstown', 'mexico_city'",
 "'mexico_city', 'canberra', 'rome'",
 "'mexico_city', 'ankara', 'rome'",
 "'mexico_city', 'lima', 'rome'",
 "'mexico_city', 'kuala_lumpur', 'rome'",
 "'mexico_city', 'london', 'rome'",
 "'mexico_city', 'copenhagen', 'rome'",
 "'mexico_city', 'cancun', 'rome'",
 "'mexico_city', 'cusco', 'rome'",
 "'mexico_city', 'bursa', 'rome'",
 "'mexico_city', 'hamburg', 'rome